<a href="https://www.kaggle.com/code/sabyasachi96/myofarm-pyimage?scriptVersionId=166375005" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# All the imports
!pip install patchify
!pip install GPUtil
!pip install torchsummary

from GPUtil import showUtilization as gpu_usage

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
# import seaborn as sns
import os
import cv2
# import random
import glob
import PIL
from PIL import Image
from tqdm import tqdm
import imghdr
from patchify import patchify 


import time
import torch
import torchvision
import torch.optim as optim
import albumentations as A
import torch.nn as nn
import torchvision.transforms.functional as TF
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torch.nn import Dropout
from torchsummary import summary

from torch.nn import BatchNorm2d 

from torchvision.transforms import CenterCrop
from torch.nn import functional as F
from torch.nn.functional import normalize

- source_tutorial: https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/
- unet: https://github.com/milesial/Pytorch-UNet
- Probablistic_unet: https://github.com/stefanknegt/Probabilistic-Unet-Pytorch/blob/master/probabilistic_unet.py
- utility script: https://github.com/CaptainDredge/Image-segmentation-utilities
- GPU_utility: https://github.com/anderskm/gputil
- torch_em: https://github.com/computational-cell-analytics/dl-for-micro/blob/main/2_cell_segmentation/torchem-train-cell-membrane-segmentation.ipynb
- torch_em: https://github.com/constantinpape/torch-em/blob/main/torch_em/model/unet.py
- digital_sreeni: https://github.com/bnsreenu/python_for_microscopists/blob/master/206_sem_segm_large_images_using_unet_with_custom_patch_inference.py
- data_Augmentatin: https://albumentations.ai/docs/getting_started/mask_augmentation/
- learning rate https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling



### loss functionds
The validation loss function is just a metric and actually not needed for training. It's there because it make sense to compare the metrics which your network is actually optimzing on. So you can add any other loss function as metric during compilation and you'll see it during training.
- https://dev.to/_aadidev/3-common-loss-functions-for-image-segmentation-545o
- https://www.kaggle.com/code/yassinealouini/all-the-segmentation-metrics/notebook
- https://github.com/JunMa11/SegLoss 
- https://pyimagesearch.com/2019/10/14/why-is-my-validation-loss-lower-than-my-training-loss/
- important for iunet loss and everytjing https://discuss.pytorch.org/t/unet-implementation/426/12

name coding: 
bSz4_1CH_ip128_vls0.2_ADM_EnDc16-512_drp_ep10_aug4

- bsz: batch size
- lR: learning rate 1e-num
- SDG: optimiser
- ADM: Adam optimiser
- clcLR: scheduler 
- vls: validation split
- EnDc: encodert decoder channels
- pt: patience for earkly stopping
- i/p : input size 
- ep: epochs
- wd: weight decay(l2 reg)
- drp: drop out added
- 1CH: input one channel instead of RGB
- aug: num of augmentation

## Setting up configuration for running in notebooks

- The pin memory is set to True to the DataLoader which will automatically put the fetched data Tensors in pinned memory, enabling faster data transfer to CUDA-enabled GPU's. For every epoch the data is transferred from CPU to GPU, with augmentations done in the CPU, and trainings done in the GPU.
- Check the output directory for saving the images, model path, plot path and test path (BASE_OUTPUT, MODEL_PATH, PLOT_PATH, TEST_PATH)
- Note keep the bacthsize small (2,5) untill you start using pacthes of the image otherwise the GPU will run out of memory

In [ ]:
# define the test split
# TEST_SPLIT = 0.3
VALIDATION_SPLIT = 0.3

# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# determine if we will be pinning memory during data loading
# PIN_MEMORY = True if DEVICE == "cuda" else False

# define the number of inputchannels in the input, number of classes,
# and number of levels (start and end) in the U-Net model
NUM_CHANNELS = 1
NUM_CLASSES = 1
NUM_LEVELS = 3
START_CHANEL = 16
END_CHANEL = 256
NUM_WORKERS = 2 if torch.cuda.is_available() else 1 

# initialize learning rate, number of epochs to train for, and the
# batch size ( in general small batch size has 256 samples, here due to 2 batch ss we have )
INIT_LR = 1e-3
NUM_EPOCHS = 10
BATCH_SIZE = 2
PATCH_SIZE = 128
WEIGHT_DECAY = 1e-6

# define the input image dimensions
# INPUT_IMAGE_WIDTH = 2048
# INPUT_IMAGE_HEIGHT = 1536

INPUT_IMAGE_WIDTH = PATCH_SIZE
INPUT_IMAGE_HEIGHT = PATCH_SIZE
NUM_AUGMENTATION = 3

#learning rate scheduler 
Patience = 4
MAX_lr = 1e-1
BASE_lr = 1e-6
STEP_SIZE = 50

# define thresholds for early stopping, for accuracy calculation and predcitions 
EARLY_STOP_THRES = 3
best_accuracy = -1
best_epoch = -1
THRESHOLD = 0.5


plotName = f"bSz{BATCH_SIZE}_ip{PATCH_SIZE}_EnDc{START_CHANEL}-{END_CHANEL}_ep{NUM_EPOCHS}_vls{VALIDATION_SPLIT}_aug{NUM_AUGMENTATION}"


BASE_OUTPUT = "/kaggle/working/"
# define the path to the output serialized model, model training
# plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, f"{plotName}")
# PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
# TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

# define the path to the base output directory
gpu_usage() 

In [ ]:
import re
name = "bSz2_ip128_EnDc16-256_ep10_vls0.3_aug3"
# Define the regular expression pattern
pattern = r"(ep\d+)"
# int(re.search(pattern, name).group(0))

# Define the regular expression pattern to only capture the digits
pattern = r"(ep\d+)"

# Extract the number and convert it to an integer
lastEpochInfo = re.search(pattern, name)[0] # Extract the entire match
int(re.search("\d+", lastEpochInfo)[0]), lastEpochInfo
# print("Extracted number:", number)

## CREATING THE CUSTOM DATASET

In [ ]:
def readData(imgPath, labelPath, convertType):
    """Reads and creates a list of target""" 
    image = []
    label = []
    imageList = []
    labelList = []
  
    for i, image_name in enumerate(sorted(os.listdir(imgPath))):
        if ((('.').join(image_name.split('.')[-1:])== 'tif') or (('.').join(image_name.split('.')[-1:]) == 'tiff')):
            label_name = '.'.join(image_name.split('.')[:-1]) +  '_bn.tif'
            
            if label_name in list(os.listdir(labelPath)): 
                # normalise by 255.0 -> convert to array -> append to list
                img_Path = os.path.join(imgPath, image_name)
                img = Image.open(img_Path).convert(convertType)
                img = np.array(img, dtype = np.float32)/255.0
                image.append(img)
                imageList.append((img_Path))
                
                label_Path = os.path.join(labelPath, label_name)
                img = Image.open(label_Path).convert(convertType)
                label.append(np.where(np.array(img) >= 1, 1.0, 0.0))
                labelList.append((label_Path))
            else:
                print('Images with no mask-->', image_name)
        else: print('Image with new extension', image_name)
    print(f'total images --> {len(imageList)}, total masks --> {len(labelList)}')       
    return image, label
            
input_folder = '/kaggle/input/myofarm/Images_input'
label_folder = '/kaggle/input/myofarm/Images_target'
image, label = readData(input_folder, label_folder, convertType = 'L')

/kaggle/input/myofarm/Images_target/20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2_bn.tif

'/kaggle/input/myofarm/Images_target/20230306_JK_230008_d12_isWT1Bld_2_261_bn.tif'
'/kaggle/input/myofarm/Images_target/20230217_JK_BRAF_mc_8_p23_d9_230007_bn.tif'

In [ ]:
def plotSanityCheckImages(img1, img2):
    
    print(f'Images:{img1.shape} and mask:{img2.shape} shape\n')
    
    figure, ax = plt.subplots(nrows=1, ncols=3, figsize=(10,10))
    ax[0].imshow(img1)
    ax[1].imshow(img2, cmap = 'gray')
    ax[2].imshow(img2 * img1, cmap = 'gray')

    ax[0].set_title("Image with the original channel")
    ax[1].set_title("Original Mask")
    ax[2].set_title("combined and made gray")

    figure.tight_layout()
    figure.show()

plotSanityCheckImages(image[0], label[0])

In [ ]:
def dataTransform(image, mask):
    images_list, masks_list = [], []
    
    transform = A.Compose([ A.HorizontalFlip(p = 0.5),
                            A.VerticalFlip(p = 0.5),
                            A.RandomBrightnessContrast(p=0.5),
                            A.ElasticTransform(p=0.5),
                            A.GridDistortion(p = 0.5), ])

    for i in range(NUM_AUGMENTATION):
        augmentations = transform(image = np.array(image), mask = np.array(mask))
        images_list.append(augmentations["image"])
        masks_list.append(augmentations["mask"])
        
    return images_list, masks_list
    
print('To check the augmentation patterns')



In [ ]:
def createPatches(imgList, maskList, PATCH_SIZE):
    images = []
    masks = []
    for i, (image, mask) in enumerate(zip(imgList, maskList)):      
        patch_images = patchify(image, (PATCH_SIZE,PATCH_SIZE), step = PATCH_SIZE)
        patch_masks = patchify(mask, (PATCH_SIZE,PATCH_SIZE), step = PATCH_SIZE)
        
        for i in range(patch_images.shape[0]):
            for j in range(patch_images.shape[1]):
                single_patch_img = patch_images[i,j,:,:]
                images.append(single_patch_img)
                
                single_patch_img = patch_masks[i,j,:,:]
                masks.append(single_patch_img)
                
    images = torch.reshape(torch.tensor(np.array(images)), [-1,1,PATCH_SIZE,PATCH_SIZE])       
    masks = torch.reshape(torch.tensor(np.array(masks)), [-1,1,PATCH_SIZE,PATCH_SIZE])
        
    return images, masks    
    

Suppose NUM_AUGMENTATION is set to 15, and you have 10 original images in your dataset. So, the total number of items in the augmented dataset would be 10 * 15 = 150.

When idx is 25:
original_idx = 25 // 15 = 1
augmentation_idx = 25 % 15 = 10
This means the current index corresponds to the 10th augmentation of the 2nd original image.
When idx is 43:
original_idx = 43 // 15 = 2
augmentation_idx = 43 % 15 = 13
This means the current index corresponds to the 13th augmentation of the 3rd original image.


he createPATCHES function likely processes multiple images and masks at once. By passing [augmented_img] and [augmented_mask] (which are lists), you're providing the function with the necessary format for handling multiple images and masks, even though in this case, you're passing only one of each.

In [ ]:
class SpheroidDataset(Dataset):

    def __init__(self, imagePaths, maskPaths, transforms = None, num_augmentations=NUM_AUGMENTATION):
        self.imagePaths = imagePaths
        self.maskPaths = maskPaths
        self.transforms = transforms
        self.num_augmentations = num_augmentations

        # Read and store image and mask data efficiently
        self.images, self.masks = readData(self.imagePaths, self.maskPaths, "L")

    def __len__(self):
        if self.transforms == True:
            length = len(self.images) * self.num_augmentations
        else:
            length = len(self.images)
        print(f'Images Augmentated-->{self.transforms}; so Dataset_length-->{length}')
        return length 
    

    def __getitem__(self, idx):
        if self.transforms == True:
            original_idx = idx // self.num_augmentations
            augmentation_idx = idx % self.num_augmentations
        else:
            original_idx = idx 

        # Retrieve image and mask from pre-loaded data
        img = self.images[original_idx]
        mask = self.masks[original_idx]

        # Transform(y/n) -> patches 
        if self.transforms == True:
            images_list, masks_list = dataTransform(img, mask)
            augmented_img = images_list[augmentation_idx]
            augmented_mask = masks_list[augmentation_idx]
            image_patch, mask_patch = createPatches([augmented_img], [augmented_mask], PATCH_SIZE)
        else:
            image_patch, mask_patch = createPatches([img], [mask], PATCH_SIZE)

        return image_patch, mask_patch 


# input_folder = '/kaggle/input/myofarm/Images_input'
# mask_folder = '/kaggle/input/myofarm/Images_target'

# dataset = SpheroidDataset(input_folder, mask_folder, transforms = True)

# for images, masks in dataset:    
#     print(images.shape) #print path and others also for checking
#     print(torch.min(images), torch.mean(images), torch.max(images))
#     print(masks.shape)
#     print('\n')
#     break
    
# print("Total images: ", len(dataset))

Images with no mask--> 20230217_JK_BRAF_mc_8_p23_d9_230007.tif
Images with no mask--> 20230306_JK_230008_d12_isWT1Bld_2_261.tif
total images --> 122
total masks --> 122
transforming
torch.Size([192, 1, 128, 128])
tensor(0.2118) tensor(0.5156) tensor(0.9176)
torch.Size([192, 1, 128, 128])


original_length-->122
augmented_length-->488
Total images:  488

## LOSS Functions and Accuracy metric


In [ ]:
def modelAccuracy(inputs, targets):
    # Calculate pixel-wise accuracy
    # assuming input and targets afe falttened
    correct = (preds == targets).float()
    accuracy = correct.sum() / targets.numel()
    
    return accuracy

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        # inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth) 
        accuracy = modelAccuracy(inputs, targets)
        
        return 1 - dice, accuracy

In [ ]:
#PyTorch
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        # inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        accuracy = modelAccuracy(inputs, targets)
        
        return Dice_BCE, accuracy

In [ ]:
#PyTorch
ALPHA = 0.8
GAMMA = 2

class FocalLoss(nn.Module):
    # loss = model_lossFunc(pred.to(torch.float32), y.to(torch.float32))
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        #comment out if your model contains a sigmoid or equivalent activation layer
        # inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1) #model_predictions
        targets = targets.view(-1) #originalMasks
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
        
        accuracy = modelAccuracy(inputs, targets)
                       
        return focal_loss, accuracy

## Building UNET

- Overall, our U-Net model will consist of an Encoder class and a Decoder class. The encoder will gradually reduce the spatial dimension to compress information. Furthermore, it will increase the number of channels, that is, the number of feature maps at each stage, enabling our model to capture different details or features in our image. On the other hand, the decoder will take the final encoder representation and gradually increase the spatial dimension and reduce the number of channels to finally output a segmentation mask of the same spatial dimension as the input image.

- Next, we define a Block module as the building unit of our encoder and decoder architecture. It is worth noting that all models or model sub-parts that we define are required to inherit from the PyTorch Module class, which is the parent class in PyTorch for all neural network modules.

## trial model with channel info outside 

In [ ]:
def UNETchannels(startChanel, endChanel, num_chanel = NUM_CHANNELS):
    startPower = int(math.log2(startChanel))
    endPower = int(math.log2(endChanel))
    enCh = [num_chanel]
    deCh = []
    for num in range(startPower, endPower+1):
        enCh.append(2**num)
    deCh = enCh[::-1]
    deCh.pop()
    return tuple(enCh), tuple(deCh)

# START_CHANEL = 16
# END_CHANEL = 1024
enCh, deCh = UNETchannels(START_CHANEL, END_CHANEL)

In [ ]:
class Block(Module):
    def __init__(self, inChannels, outChannels):
        super().__init__()
        self.conv1 = nn.Conv2d(inChannels, outChannels,
                            kernel_size=3, stride=1, padding=1, bias=True)
        self.BN1 = nn.BatchNorm2d(outChannels)
        self.relu1 = nn.ReLU(inplace = True)
        self.dropout = nn.Dropout(0.25)
            
        self.conv2 = Conv2d(outChannels, outChannels, 
                            kernel_size=3, stride=1, padding=1, bias=True)
        self.BN2 = BatchNorm2d(outChannels)
        self.relu2 = ReLU(inplace = True)
        
    def forward(self, x):
        # apply CONV => [BN] => RELU => CONV block to the inputs and return it
        outputConv1 = self.dropout(self.relu1(self.BN1(self.conv1(x))))
        outputConv2 = self.relu2(self.BN2(self.conv2(outputConv1))) 
        return outputConv2

In [ ]:
class Encoder(Module):    
    def __init__(self, channels = enCh): 
        super().__init__()
        self.encBlocks = ModuleList([Block(channels[i], channels[i + 1]) 
                                      for i in range(len(channels) - 1)])
        self.pool = MaxPool2d(kernel_size=2, stride=2)

    # this takes an input image x
    # initialize an empty list (blockOutputs) to store the intermediate outputs passed
    # to the decoder where they can be processed with the decoder feature maps.   
    def forward(self, x): 
        blockOutputs = []
        for block in self.encBlocks:
            x = block(x)
            blockOutputs.append(x)
            x = self.pool(x)

        return blockOutputs
    
    
class Decoder(Module):
    def __init__(self, channels = deCh):
        super().__init__()

        self.channels = channels
        self.upconvs = ModuleList([ConvTranspose2d(channels[i], channels[i + 1], kernel_size=2, stride=2)
                                   for i in range(len(channels) - 1)])
        self.dec_blocks = ModuleList([Block(channels[i], channels[i + 1])
                                       for i in range(len(channels) - 1)])

    # Input is the feature map x and 
    # the list of intermediate outputs from the encoder 
    def forward(self, x, encFeatures):

        for i in range(len(self.channels) - 1):
            x = self.upconvs[i](x)
            # crop the current features from the encoder blocks,
            # concatenate them with the current upsampled features,
            # and pass the concatenated output through the current decoder block
            # encFeat = self.crop(encFeatures[i], x)
            x = torch.cat([x, encFeatures[i]], dim=1)
            x = self.dec_blocks[i](x)

        return x  

class UNet(Module):
        
    def __init__(self, encChannels = enCh,
                 decChannels = deCh, 
                 nbClasses=1, retainDim=True,
                 outSize=(INPUT_IMAGE_HEIGHT,  INPUT_IMAGE_WIDTH)):
        super().__init__()
        self.encoder = Encoder(encChannels)
        self.decoder = Decoder(decChannels)
        self.head = nn.Sequential(nn.Conv2d(decChannels[-1], nbClasses, kernel_size=1),
                                  nn.Sigmoid() )
        self.retainDim = retainDim
        self.outSize = outSize
        
        
    def forward(self, x):
        # grab the features from the encoder
        # Note that the encFeatures list contains 
        # all the feature maps starting from the first encoder block output to the last
        encFeatures = self.encoder(x)
        # pass the encoder features through decoder making sure that
        # their dimensions are suited for concatenation
        # since the encoder feature maps starting from the last encoder block output to the first of the decoder
       
        # output of the final encoder block 
        # (i.e., encFeatures[::-1][0]) and the feature map outputs of all intermediate encoder blocks 
        # (i.e., encFeatures[::-1][1:]) to the decoder
        decFeatures = self.decoder(encFeatures[::-1][0], encFeatures[::-1][1:])
        map = self.head(decFeatures)
        
        if self.retainDim:
            map = F.interpolate(map, self.outSize)

        return map   
summary(UNet().to(DEVICE), input_size=(1,PATCH_SIZE,PATCH_SIZE))

- nbClasses: This defines the number of segmentation classes where we have to classify each pixel. This usually corresponds to the number of channels in our output segmentation map, where we have one channel for each class. Since we are working with two classes (i.e., binary classification), we keep a single channel and use thresholding for classification, as we will discuss later.

- Now the encFeatures[::-1] list contains the feature map outputs in reverse order (i.e., from the last to the first encoder block). Note that this is important since, on the decoder side, we will be utilizing the encoder feature maps starting from the last encoder block output to the first.

## load previously trained model


In [ ]:
def loadOldModel(path, newModel = None):
    if newModel == True:
        

In [ ]:
import os #/kaggle/input/test_myotwin/pytorch/unet-bsz2_ip128_endc16-256_ep10/1/unet_spheroid-3.pth
os.listdir("/kaggle/input/test_myotwin/pytorch/v2fromver56/2") #/kaggle/input/test_myotwin/pytorch/v2fromver56/2
os.list("/kaggle/input/test_myotwin/pytorch/v2fromver56/2/bSz2_ip128_EnDc16-256_ep10_vls0.3_aug3_FullModel.pth")

## checkpoints and saving of model 

In [ ]:
# this is for the full model saving 
def save_checkpoint(state, filename):
    print("=> Saving complete model ")
    filename = filename + '_FullModel.pth'
    torch.save(state, filename)
    return filename
    
# this is just to save the model parameters not very comprehensive
def checkpoint_inference(model, epoch, optimizer, filename):
    print("=> Saving all model parameters ")
    filename = filename + '_ModelInference.pth'
    torch.save({
            'model': unet,
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, filename)

## Train the UNET model

- ToPILImage(): it enables us to convert our input images to PIL image format. Note that this is necessary since we used OpenCV to load images in our custom dataset, but PyTorch expects the input image samples to be in PIL format.
- Resize(): allows us to resize our images to a particular input dimension (i.e., config.INPUT_IMAGE_HEIGHT, config.INPUT_IMAGE_WIDTH) that our model can accept
- ToTensor(): enables us to convert input images to PyTorch tensors and convert the input PIL Image, which is originally in the range from [0, 255], to [0, 1].


In [ ]:
# BATCH_SIZE = 5
# VALIDATION_SPLIT = 0.2
train_dataset = SpheroidDataset(input_folder, label_folder, transforms = True)
train_set, validation_set = torch.utils.data.random_split(train_dataset, 
                                                          [round(len(train_dataset)*(1-VALIDATION_SPLIT)), 
                                                           round(len(train_dataset)*VALIDATION_SPLIT)])

trainLoader = DataLoader(dataset = train_set, batch_size = BATCH_SIZE, num_workers = NUM_WORKERS, shuffle =True)
validationLoader = DataLoader(dataset = validation_set, batch_size = BATCH_SIZE, num_workers = NUM_WORKERS, shuffle =True) 

# calculate steps per epoch for training and test set
trainSteps = len(train_set) // BATCH_SIZE
testSteps = len(validation_set) // BATCH_SIZE
validationSteps = len(validation_set) // BATCH_SIZE

print(f"Total {len(train_set)} instances in the training and {trainSteps} per epoch")
print(f"Total {len(validation_set)} instances in the validation set and {validationSteps} per epoch")
print(f"Each Epoch runs: {len(trainLoader)} times, where each loader has {BATCH_SIZE} instances for trainig")

gpu_usage() 

In [ ]:
import re
name = "bSz2_ip128_EnDc16-256_ep10_vls0.3_aug3"
# Define the regular expression pattern
pattern = r"(ep\d+)"
# int(re.search(pattern, name).group(0))

# Define the regular expression pattern to only capture the digits
pattern = r"(ep\d+)"

# Extract the number and convert it to an integer
lastEpochInfo = re.search(pattern, name)[0] # Extract the entire match
int(re.search("\d+", lastEpochInfo)[0]), lastEpochInfo
# print("Extracted number:", number)

In [ ]:
oldModel = str(os.listdir('/kaggle/input/test_myotwin/pytorch'))
pattern = r"(ep\d+)"
trainedEpoch = re.search(pattern, oldModel)[0]
trainedEpoch = re.search("\d+", lastEpochInfo)[0]
print(trainedEpoch)

In [ ]:
newModel = True
path = '/kaggle/input/test_myotwin/pytorch/unet-bsz2_ip128_endc16-256_ep10/1/unet_spheroid-3.pth'
if newModel == True:
#     unet = torch.load(path).to(DEVICE)
    print('Model Loaded')

In [ ]:

# initialize loss function and optimizerimizer
metric_lossFunc = DiceBCELoss() #nn.BCELoss() #DiceLoss()
optimizer = optim.Adam(unet.parameters(), lr= INIT_LR, weight_decay = WEIGHT_DECAY)
# optimizer = torch.optim.SGD(unet.parameters(), lr=INIT_LR, momentum=0.9, weight_decay = WEIGHT_DECAY)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor= 0.1, patience=Patience, verbose = True)
# scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1, cycle_momentum = True, verbose = True)

# initialize a dictionary to store training history
H = {"train_loss": [], "validation_loss": [], "model_accuracy":[]}



In [ ]:
x = 0.1234567890
print(f"{x:.2f}")

In [ ]:
# loop over epochs
print("[INFO] training the network...")
startTime = time.time()
for epoch in tqdm(range(NUM_EPOCHS)):
    # set the model in training mode - initialise train and val loss - loop on train set
    unet.train()
    totalTrainLoss = 0
    totalValidationLoss_model = 0
    totalValidationLoss_metric = 0
    totalAccuracy = 0
    print(f"the model runs -> {len(trainLoader)} per epoch {epoch}")
    
    for (i, (x, y)) in enumerate(trainLoader):
        # print(f"\ntrainLoader enumerated #_{i}_section\n")
        # send the input to the device - forward pass - train loss - zero out prev gradients 
        # - back propagation - update model params - add train loss (.item makes it float)
        #""" Note make sure to change the i/p channel to later on ([-1, 1, patch_size, patch_size])"""
        x = torch.reshape(x,[-1,1,PATCH_SIZE,PATCH_SIZE]).float()
        y = torch.reshape(y,[-1,1,PATCH_SIZE,PATCH_SIZE]).float()
        (x, y) = (x.to(DEVICE), y.to(DEVICE))
        #print(x.shape)
        pred = unet(x) 
        loss, accuracy = metric_lossFunc(pred.to(torch.float32), y.to(torch.float32))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        totalTrainLoss += loss.item()
        
    # switch off autograd for validation set
    with torch.no_grad():
        unet.eval()
        for (x, y) in validationLoader:
            """ Note make sure to change the i/p channel to later on ([-1, 1, patch_size, patch_size])"""
            x = torch.reshape(x,[-1,1,PATCH_SIZE,PATCH_SIZE]).float()
            y = torch.reshape(y,[-1,1,PATCH_SIZE,PATCH_SIZE]).float()
            (x, y) = (x.to(DEVICE), y.to(DEVICE))
            pred = unet(x)
            loss, acc = metric_lossFunc(pred.to(torch.float32), y.to(torch.float32)).item()
            totalValidationLoss_metric += loss
            totalAccuracy += acc
            
    # calculate the average training and validation loss and binary accuracy
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValidationLoss = totalValidationLoss_metric / validationSteps
    avgAcc = totalAccuracy / validationSteps
    
    H["train_loss"].append(avgTrainLoss)
    H["validation_loss"].append(avgValidationLoss) 
    H["model_accuracy"].append(avgAcc) 
    
    # print the model training and validation information
    print(f"For EPOCH: {epoch + 1}/{NUM_EPOCHS}, and model goes through data->{len(trainLoader)} times)")
    print(f"Train loss: {avgTrainLoss:.6f}, Validation loss: {avgValidationLoss:.4f},Val accuracy: {avgAcc:.4f}%")
    gpu_usage()
    
    # saving the intermidiary informations
    checkpoint_iference(model, epoch, optimizer, filename, newModel = None)

def checkpoint_inference(model, epoch, optimizer, filename, newModel = None):
    
    if newModel == True:
          totalEpochs = 
          
    print("=> Saving all model parameters ")
    filename = filename + '_ModelInference.pth'
    torch.save({
            'model': unet,
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, filename)

    # terminate the training loop
#     if avgAcc > best_accuracy:
#         best_accuracy = avgAcc
#         best_epoch = epoch
#         checkpoint(unet, MODEL_PATH)
#     elif epoch - best_epoch > EARLY_STOP_THRES:
#         print("Early stopped training at epoch %d" % epoch)
#         break  
    
    # Step the scheduler based on training loss 
#     scheduler.step(avgTrainLoss)
    # Step the scheduler based on validation loss 
#     scheduler.step(avgValidationLoss)
    
# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

In [ ]:
# save the final model and check_point inference if the mdel training completes with no issues
MODEL_PATH = save_checkpoint(unet, MODEL_PATH)
checkpoint_inference(unet,  epoch, optimizer, MODEL_PATH)

In [ ]:

plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["validation_loss"], label="validation_loss")
plt.plot(H["model_accuracy"], label="accuracy")
plt.title(f"Losses for version:-\n{plotName}")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")



# Model used for prediction of segemnted images

In [ ]:

def prepare_plot(origImage, origMask, predMask, file, threshold = None):
    # initialize our figure
    figure, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 15))
    ax[0].imshow(origImage)
    ax[1].imshow(origMask)
    ax[2].imshow(predMask)
#     ax[3].imshow(predMask > (predMask.max() - threshold))
#     ax[3].hist(predMask.flatten()*255)

    ax[0].set_title(f"{file}")
    ax[1].set_title("Original Mask")
    ax[2].set_title(f"P_Mask-{threshold}thresh")
#     ax[3].set_title('P_mask w threshold')

    figure.tight_layout()
    figure.show()

In [ ]:
def make_predictions(model, file, imagePath, groundTruthPath, patch_size, threshold = None):
    model.eval()
    with torch.no_grad():
        image = Image.open(imagePath).convert("L")
        image = np.float32(image)/255.0
        
        gtMask = np.array(Image.open(groundTruthPath).convert("L"))
        gtMask[gtMask > 0] = 1.0
    
        height, width = image.shape[:2]
        segm_img = np.zeros((height, width), dtype=np.uint8)  # Array with zeros to be filled with segmented values
    
        patch_num = 1
    
        for i in range(0, height, patch_size):  
            for j in range(0, width, patch_size):  
                single_patch = image[i:i+patch_size, j:j+patch_size]
                single_patch = np.expand_dims(np.expand_dims(single_patch, 0), 0)
                orig_patch = single_patch.copy()
                single_patch = torch.from_numpy(single_patch).to(DEVICE)
                
                Mask_patch = gtMask[i:i+patch_size, j:j+patch_size]
               
                # pass the results through the sigmoid if the last layer of the model doesnot do sigmoid conversion
                # single_patch_prediction = torch.sigmoid(model(single_patch)).squeeze().cpu().numpy()
                single_patch_prediction = model(single_patch).squeeze().cpu().numpy()
                
                # filter out the weak predictions and convert them to integers
                single_patch_prediction = ((single_patch_prediction>threshold)*255).astype(np.uint8)
                
                single_patch_shape = single_patch_prediction.shape[:2]
                segm_img[i:i+single_patch_shape[0], j:j+single_patch_shape[1]] += cv2.resize(single_patch_prediction, single_patch_shape[::-1])
                # print("Finished processing patch number ", patch_num, " at position ", i, j)
            
            patch_num += 1  
    # print(f"Finised-{file}")
    prepare_plot(image, gtMask, segm_img, file, threshold)
    return single_patch_prediction

In [ ]:

print("[INFO] load up model...")

print("[INFO] loading up test image paths...\n")
test_input = '/kaggle/input/myofarm/test_inputs'
test_mask = '/kaggle/input/myofarm/test_masks'

files = os.listdir(test_input)
for file in files:
    imagePath = test_input+ '/' + file
    groundTruthPath = test_mask + '/' + '.'.join(file.split('.')[:-1]) + '_bn.tif'
    simg = make_predictions(unet, file, imagePath, groundTruthPath, PATCH_SIZE, threshold = 0.5)



In [ ]:

# imagePaths = []


# test_input = '/kaggle/input/myofarm/test_inputs'
# test_mask = '/kaggle/input/myofarm/test_masks'
# files = np.random.choice(os.listdir(test_input), size = 4)
# for file in files:
#     imagePath = test_input+ '/' + file
#     image = np.array(Image.open(imagePath).convert("L"))
#     plt.imshow(image)
#     plt.show()
#     groundTruthPath = test_mask + '/' + '.'.join(file.split('.')[:-1]) + '_bn.tif'
#     image = np.array(Image.open(groundTruthPath).convert("L"))
#     image[image > 0] = 1.0
#     plt.imshow(image)
#     plt.show()
#     print(imagePath, groundTruthPath)